In [1]:
import pandas as pd
from tqdm import tqdm
from nixtla import NixtlaClient

In [2]:
nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-kANBNrQA7ZnmSrU61fLxQiT6bJOPcMVOiADzNw5AYY8dpQdv7FKdhdvV6JHP8sJtKaNMU8k3h5RNUZmz' # yzy api
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [3]:
start_point = 3000*12
batch_size = 12
num = 1000
output_file = './data/forecasts_3001_4000.csv'

flatten_df = pd.read_csv('./data/flattened_loop_sensor_test_12h.csv')
flatten_df = flatten_df.iloc[start_point :start_point + batch_size * num]
flatten_df['t_1h'] = pd.to_datetime(flatten_df['t_1h'])
print(flatten_df.shape)
flatten_df.head()

(12000, 5)


,id,iu_ac,t_1h,etat_barre,q
36000,3001,21,2023-06-29 08:00:00,2,292.0
36001,3001,21,2023-06-29 09:00:00,2,241.0
36002,3001,21,2023-06-29 10:00:00,2,203.0
36003,3001,21,2023-06-29 11:00:00,2,201.0
36004,3001,21,2023-06-29 12:00:00,2,209.0


In [4]:
num_batches = len(flatten_df) // batch_size
all_forecasts = []

In [5]:
# Forecasting loop
for i in tqdm(range(num_batches), desc='Forecasting'):
    # 读取12条数据
    df = flatten_df.iloc[i * batch_size:(i + 1) * batch_size]
    
    # 进行预测
    timegpt_fcst_df = nixtla_client.forecast(
        df=df, 
        h=1,  # 预测步长
        # finetune_steps=10,
        # finetune_loss='mae',   
        freq='h', 
        time_col='t_1h', 
        target_col='q', 
        model='timegpt-1'
    )

    # Add index starting from 1 + start_point
    timegpt_fcst_df.reset_index(drop=True, inplace=True)
    timegpt_fcst_df.index += 1 + int(start_point/12) + i

    # Save the current forecast to the output CSV file
    if i == 0:
        # For the first batch, write the header
        timegpt_fcst_df.to_csv(output_file, index=True, index_label='id', mode='w')
    else:
        # For subsequent batches, append without the header
        timegpt_fcst_df.to_csv(output_file, index=True, index_label='id', mode='a', header=False)

    # Append the current forecast to the list
    all_forecasts.append(timegpt_fcst_df)

# Optionally, you can still combine all forecasts into one DataFrame and perform additional operations if needed
all_forecasts_df = pd.concat(all_forecasts, ignore_index=False)
print(all_forecasts_df.shape)
print(all_forecasts_df.head())


Forecasting:   0%|          | 0/1000 [00:00<?, ?it/s]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 1/1000 [00:04<1:23:03,  4.99s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 2/1000 [00:08<1:08:43,  4.13s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
Forecasting:   0%|          | 3/1000 [00:13<1:12:12,  4.35s/it]INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
I

(1000, 2)
                      t_1h     TimeGPT
36001  2023-06-29 20:00:00  203.163254
36013  2023-07-01 01:00:00  101.744736
36025  2023-07-02 06:00:00   75.104340
36037  2023-07-03 11:00:00  212.581253
36049  2023-07-04 16:00:00  200.820877
